1)	**«Gesture Phase Segmentation»**

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00302/gesture_phase_dataset.zip

--2025-02-23 11:04:35--  https://archive.ics.uci.edu/ml/machine-learning-databases/00302/gesture_phase_dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘gesture_phase_dataset.zip.1’

gesture_phase_datas     [   <=>              ]   1.85M  3.91MB/s    in 0.5s    

2025-02-23 11:04:36 (3.91 MB/s) - ‘gesture_phase_dataset.zip.1’ saved [1943494]



In [27]:
import numpy as np
import pandas as pd
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Run on CPU

import tensorflow as tf
import argparse

In [ ]:
train_traces = []
valid_traces = []
test_traces = []

In [ ]:
def load_trace(filename):
    df = pd.read_csv(filename,header=0)

    str_y = df["Phase"].values
    convert = {"D":0,"P":1,"S":2,"H":3,"R":4}
    y = np.empty(str_y.shape[0],dtype=np.int32)
    for i in range(str_y.shape[0]):
        y[i] = convert[str_y[i]]

    x = df.values[:,:-1].astype(np.float32)

    return (x,y)

def cut_in_sequences(tup,seq_len,interleaved=False):
    x,y = tup

    num_sequences = x.shape[0]//seq_len
    sequences = []

    for s in range(num_sequences):
        start = seq_len*s
        end = start+seq_len
        sequences.append((x[start:end],y[start:end]))

        if(interleaved and s < num_sequences - 1):
            start += seq_len//2
            end = start+seq_len
            sequences.append((x[start:end],y[start:end]))

    return sequences


In [ ]:
df = pd.read_csv('/content/a3_va3.csv')
df

,1,2,3,4,5,6,7,8,9,10,...,24,25,26,27,28,29,30,31,32,Phase
0,-0.000010,-0.000021,0.000005,-0.000006,0.000043,-0.000011,0.000004,-0.000027,0.000006,-2.852000e-05,...,-1.770000e-06,0.000024,0.000045,0.000028,0.000084,0.000003,0.000010,0.000004,0.000015,D
1,-0.000006,-0.000017,0.000004,-0.000020,0.000064,-0.000010,0.000008,-0.000023,0.000005,-4.526000e-05,...,-1.770000e-06,0.000019,0.000068,0.000025,0.000154,0.000002,0.000008,0.000002,0.000021,D
2,-0.000025,-0.000026,0.000004,-0.000027,0.000061,-0.000008,-0.000005,-0.000030,0.000007,-5.296000e-05,...,9.100000e-07,0.000036,0.000067,0.000031,0.000154,0.000006,0.000003,0.000005,0.000003,D
3,-0.000041,-0.000037,0.000005,-0.000033,0.000060,-0.000006,-0.000020,-0.000038,0.000009,-5.116000e-05,...,1.660000e-06,0.000055,0.000069,0.000044,0.000127,0.000006,0.000002,0.000005,0.000008,D
4,-0.000052,-0.000046,0.000005,-0.000030,0.000044,-0.000004,-0.000029,-0.000045,0.000010,-3.944000e-05,...,2.100000e-06,0.000069,0.000054,0.000054,0.000068,0.000004,0.000005,0.000003,0.000018,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1825,0.000455,-0.000141,-0.000153,0.000054,-0.000193,-0.000097,0.000749,-0.000195,-0.000254,4.191000e-05,...,7.000000e-06,0.000500,0.000222,0.000815,0.000353,0.000122,0.000020,0.000102,0.000027,D
1826,0.000219,-0.000073,-0.000124,0.000058,-0.000168,-0.000070,0.000441,-0.000103,-0.000196,4.712000e-05,...,5.790000e-06,0.000262,0.000191,0.000494,0.000292,0.000062,0.000009,0.000082,0.000016,D
1827,0.000080,-0.000060,-0.000115,0.000039,-0.000176,-0.000058,0.000226,-0.000069,-0.000165,2.376000e-05,...,2.730000e-06,0.000153,0.000189,0.000288,0.000285,0.000042,0.000007,0.000067,0.000008,D
1828,-0.000021,-0.000067,-0.000111,0.000034,-0.000185,-0.000048,0.000054,-0.000065,-0.000143,8.810000e-06,...,2.030000e-06,0.000132,0.000194,0.000166,0.000292,0.000031,0.000005,0.000054,0.000006,D


**Подготовка данных**

In [ ]:
interleaved_train = True
training_files = ['/content/a3_va3.csv']
for f in training_files:
    train_traces.extend(cut_in_sequences(load_trace(os.path.join("data/gesture",f)),seq_len=32,interleaved=interleaved_train))

In [ ]:
train_x,train_y = list(zip(*train_traces))

train_x = np.stack(train_x,axis=1)
train_y = np.stack(train_y,axis=1)

train_x

array([[[-9.920000e-06, -2.135000e-05,  4.830000e-06, ...,
          9.560000e-06,  4.050000e-06,  1.483000e-05],
        [-1.301000e-05, -2.870000e-05, -2.500000e-07, ...,
          5.500000e-07,  7.330000e-06,  8.900000e-07],
        [-3.136500e-04, -8.890000e-06,  1.165000e-05, ...,
          5.704000e-05,  7.000000e-07,  4.142000e-05],
        ...,
        [-4.833000e-05, -3.138700e-04,  3.389000e-05, ...,
          1.867000e-05,  3.596700e-04,  1.862000e-05],
        [-1.452400e-04,  2.426600e-04, -2.037000e-05, ...,
          1.111000e-05,  2.626000e-05,  9.352000e-05],
        [ 1.848800e-04,  1.824300e-04, -3.492000e-05, ...,
          3.800000e-06,  1.023000e-05,  4.070000e-06]],

       [[-6.480000e-06, -1.739000e-05,  4.110000e-06, ...,
          7.500000e-06,  1.850000e-06,  2.102000e-05],
        [-2.717000e-05, -3.724000e-05,  2.900000e-07, ...,
          8.500000e-07,  5.330000e-06,  2.810000e-06],
        [-2.890700e-04,  3.000000e-08,  1.000000e-05, ...,
          5.48

In [ ]:
train_y

array([[0, 0, 0, ..., 3, 3, 3],
       [0, 0, 0, ..., 3, 3, 3],
       [0, 0, 0, ..., 3, 3, 3],
       ...,
       [0, 0, 1, ..., 3, 4, 0],
       [0, 0, 1, ..., 3, 4, 0],
       [0, 0, 1, ..., 3, 4, 0]], dtype=int32)

In [ ]:
flat_x = train_x.reshape([-1,train_x.shape[-1]])
mean_x = np.mean(flat_x,axis=0)
std_x = np.std(flat_x,axis=0)
train_x = (train_x-mean_x)/std_x

total_seqs = train_x.shape[1]
print("Total number of training sequences: {}".format(total_seqs))
permutation = np.random.RandomState(23489).permutation(total_seqs)
valid_size = int(0.1*total_seqs)
test_size = int(0.15*total_seqs)

valid_x = train_x[:,permutation[:valid_size]]
valid_y = train_y[:,permutation[:valid_size]]
test_x = train_x[:,permutation[valid_size:valid_size+test_size]]
test_y = train_y[:,permutation[valid_size:valid_size+test_size]]
train_x = train_x[:,permutation[valid_size+test_size:]]
train_y = train_y[:,permutation[valid_size+test_size:]]

Total number of training sequences: 113


**2)	«Occupancy Detection Dataset»**

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00357/occupancy_data.zip

--2025-02-23 11:28:03--  https://archive.ics.uci.edu/ml/machine-learning-databases/00357/occupancy_data.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘occupancy_data.zip’

occupancy_data.zip      [  <=>               ] 327.84K   976KB/s    in 0.3s    

2025-02-23 11:28:03 (976 KB/s) - ‘occupancy_data.zip’ saved [335713]



In [ ]:
df = pd.read_csv('/content/datatraining.txt')
df

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
1,2015-02-04 17:51:00,23.18,27.2720,426.0,721.250000,0.004793,1
2,2015-02-04 17:51:59,23.15,27.2675,429.5,714.000000,0.004783,1
3,2015-02-04 17:53:00,23.15,27.2450,426.0,713.500000,0.004779,1
4,2015-02-04 17:54:00,23.15,27.2000,426.0,708.250000,0.004772,1
5,2015-02-04 17:55:00,23.10,27.2000,426.0,704.500000,0.004757,1
...,...,...,...,...,...,...,...
8139,2015-02-10 09:29:00,21.05,36.0975,433.0,787.250000,0.005579,1
8140,2015-02-10 09:29:59,21.05,35.9950,433.0,789.500000,0.005563,1
8141,2015-02-10 09:30:59,21.10,36.0950,433.0,798.500000,0.005596,1
8142,2015-02-10 09:32:00,21.10,36.2600,433.0,820.333333,0.005621,1


**Подготовка данных**

In [ ]:
data_x = np.stack([
    df['Temperature'].values,
    df['Humidity'].values,
    df['Light'].values,
    df['CO2'].values,
    df['HumidityRatio'].values,
    ],axis=-1)
data_y = df['Occupancy'].values.astype(np.int32)

In [ ]:
def cut_in_sequences(x,y,seq_len,inc=1):

    sequences_x = []
    sequences_y = []

    for s in range(0,x.shape[0] - seq_len,inc):
        start = s
        end = start+seq_len
        sequences_x.append(x[start:end])
        sequences_y.append(y[start:end])

    return np.stack(sequences_x,axis=1),np.stack(sequences_y,axis=1)

In [ ]:
train_x = data_x
train_y = data_y

mean_x = np.mean(train_x,axis=0)
std_x = np.std(train_x,axis=0)
train_x = (train_x-mean_x)/std_x

seq_len=16

train_x,train_y = cut_in_sequences(train_x,train_y,seq_len)

print("Total number of training sequences: {}".format(train_x.shape[1]))
permutation = np.random.RandomState(893429).permutation(train_x.shape[1])
valid_size = int(0.1*train_x.shape[1])
print("Validation split: {}, training split: {}".format(valid_size,train_x.shape[1]-valid_size))

valid_x = train_x[:,permutation[:valid_size]]
valid_y = train_y[:,permutation[:valid_size]]
train_x = train_x[:,permutation[valid_size:]]
train_y = train_y[:,permutation[valid_size:]]

Total number of training sequences: 8127
Validation split: 812, training split: 7315


**3)	«Human Activity Recognition Using Smartphones Dataset»**

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip

--2025-02-23 11:44:15--  https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘UCI HAR Dataset.zip’

UCI HAR Dataset.zip     [           <=>      ]  58.17M  26.9MB/s    in 2.2s    

2025-02-23 11:44:18 (26.9 MB/s) - ‘UCI HAR Dataset.zip’ saved [60999314]



In [ ]:
train_x = np.loadtxt("/content/X_train.txt")
train_y = (np.loadtxt("/content/y_train.txt")-1).astype(np.int32)

In [ ]:
def cut_in_sequences(x,y,seq_len,inc=1):

    sequences_x = []
    sequences_y = []

    for s in range(0,x.shape[0] - seq_len,inc):
        start = s
        end = start+seq_len
        sequences_x.append(x[start:end])
        sequences_y.append(y[start:end])

    return np.stack(sequences_x,axis=1),np.stack(sequences_y,axis=1)


seq_len=16

train_x,train_y = cut_in_sequences(train_x,train_y,seq_len)
print("Total number of training sequences: {}".format(train_x.shape[1]))
permutation = np.random.RandomState(893429).permutation(train_x.shape[1])
valid_size = int(0.1*train_x.shape[1])
print("Validation split: {}, training split: {}".format(valid_size,train_x.shape[1]-valid_size))

valid_x = train_x[:,permutation[:valid_size]]
valid_y = train_y[:,permutation[:valid_size]]
train_x = train_x[:,permutation[valid_size:]]
train_y = train_y[:,permutation[valid_size:]]

Total number of training sequences: 7336
Validation split: 733, training split: 6603


**4)	«Metro Interstate Traffic Volume»**

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00492/Metro_Interstate_Traffic_Volume.csv.gz

--2025-02-23 15:09:19--  https://archive.ics.uci.edu/ml/machine-learning-databases/00492/Metro_Interstate_Traffic_Volume.csv.gz
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘Metro_Interstate_Traffic_Volume.csv.gz’

Metro_Interstate_Tr     [   <=>              ] 395.87K   982KB/s    in 0.4s    

2025-02-23 15:09:24 (982 KB/s) - ‘Metro_Interstate_Traffic_Volume.csv.gz’ saved [405373]



In [28]:
df = pd.read_csv("/content/Metro_Interstate_Traffic_Volume.csv")
df

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,NaN,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545
1,NaN,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516
2,NaN,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767
3,NaN,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026
4,NaN,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918
...,...,...,...,...,...,...,...,...,...
48199,NaN,283.45,0.0,0.0,75,Clouds,broken clouds,2018-09-30 19:00:00,3543
48200,NaN,282.76,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 20:00:00,2781
48201,NaN,282.73,0.0,0.0,90,Thunderstorm,proximity thunderstorm,2018-09-30 21:00:00,2159
48202,NaN,282.09,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 22:00:00,1450


In [29]:
import datetime as dt

In [30]:
holiday = (df["holiday"].values == None).astype(np.float32)
temp = df["temp"].values.astype(np.float32)
temp -= np.mean(temp)  # normalize temp by annual mean
rain = df["rain_1h"].values.astype(np.float32)
snow = df["snow_1h"].values.astype(np.float32)
clouds = df["clouds_all"].values.astype(np.float32)
date_time = df["date_time"].values
# 2012-10-02 13:00:00
date_time = [dt.datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in date_time]
weekday = np.array([d.weekday() for d in date_time]).astype(np.float32)
noon = np.array([d.hour for d in date_time]).astype(np.float32)
noon = np.sin(noon * np.pi / 24)

features = np.stack([holiday, temp, rain, snow, clouds, weekday, noon], axis=-1)

traffic_volume = df["traffic_volume"].values.astype(np.float32)
traffic_volume -= np.mean(traffic_volume)  # normalize
traffic_volume /= np.std(traffic_volume)  # normalize

In [31]:
def cut_in_sequences(x, y, seq_len, inc=1):

    sequences_x = []
    sequences_y = []

    for s in range(0, x.shape[0] - seq_len, inc):
        start = s
        end = start + seq_len
        sequences_x.append(x[start:end])
        sequences_y.append(y[start:end])

    return np.stack(sequences_x, axis=1), np.stack(sequences_y, axis=1)


seq_len=32


train_x, train_y = cut_in_sequences(features, traffic_volume, seq_len, inc=4)

train_x = np.stack(train_x, axis=0)
train_y = np.stack(train_y, axis=0)
total_seqs = train_x.shape[1]
print("Total number of training sequences: {}".format(total_seqs))
permutation = np.random.RandomState(23489).permutation(total_seqs)
valid_size = int(0.1 * total_seqs)
test_size = int(0.15 * total_seqs)

valid_x = train_x[:, permutation[:valid_size]]
valid_y = train_y[:, permutation[:valid_size]]
test_x = train_x[:, permutation[valid_size : valid_size + test_size]]
test_y = train_y[:, permutation[valid_size : valid_size + test_size]]
train_x = train_x[:, permutation[valid_size + test_size :]]
train_y = train_y[:, permutation[valid_size + test_size :]]

Total number of training sequences: 12043


In [45]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/ozone/eighthr.names

--2025-02-23 15:45:10--  https://archive.ics.uci.edu/ml/machine-learning-databases/ozone/eighthr.names
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘eighthr.names’

eighthr.names           [ <=>                ]   1.65K  --.-KB/s    in 0s      

2025-02-23 15:45:15 (382 MB/s) - ‘eighthr.names’ saved [1694]



In [32]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/ozone/eighthr.data

--2025-02-23 15:30:46--  https://archive.ics.uci.edu/ml/machine-learning-databases/ozone/eighthr.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘eighthr.data’

eighthr.data            [    <=>             ] 799.15K   441KB/s    in 1.8s    

2025-02-23 15:30:51 (441 KB/s) - ‘eighthr.data’ saved [818329]



In [41]:
def to_float(v):
    if(v == "?"):
        return 0
    else:
        return float(v)

def load_trace():
    all_x = []
    all_y = []

    with open("/content/eighthr.data","r") as f:
        miss = 0
        total = 0
        while True:
            line = f.readline()
            if(line is None):
                break
            line = line[:-1]
            parts = line.split(',')

            total+=1
            for i in range(1,len(parts)-1):
                if(parts[i]=="?"):
                    miss+=1
                    break

            if(len(parts)!=74):
                break
            label = int(float(parts[-1]))
            feats = [to_float(parts[i]) for i in range(1,len(parts)-1)]

            all_x.append(np.array(feats))
            all_y.append(label)
    print("Missing features in {} out of {} samples ({:0.2f})".format(miss,total,100*miss/total))
    print("Read {} lines".format(len(all_x)))
    all_x = np.stack(all_x,axis=0)
    all_y = np.array(all_y)

    print("Imbalance: {:0.2f}%".format(100*np.mean(all_y)))
    all_x -= np.mean(all_x) #normalize
    all_x /= np.std(all_x) #normalize

    return all_x,all_y


def cut_in_sequences(x,y,seq_len,inc=1):

    sequences_x = []
    sequences_y = []

    for s in range(0,x.shape[0] - seq_len,inc):
        start = s
        end = start+seq_len
        sequences_x.append(x[start:end])
        sequences_y.append(y[start:end])

    return np.stack(sequences_x,axis=1),np.stack(sequences_y,axis=1)

In [42]:
x,y = load_trace()

train_x,train_y = cut_in_sequences(x,y,seq_len,inc=4)

train_x = np.stack(train_x,axis=1)
train_y = np.stack(train_y,axis=1)

total_seqs = train_x.shape[1]
print("Total number of training sequences: {}".format(total_seqs))
permutation = np.random.RandomState(23489).permutation(total_seqs)
valid_size = int(0.1*total_seqs)
test_size = int(0.15*total_seqs)

valid_x = train_x[:,permutation[:valid_size]]
valid_y = train_y[:,permutation[:valid_size]]
test_x = train_x[:,permutation[valid_size:valid_size+test_size]]
test_y = train_y[:,permutation[valid_size:valid_size+test_size]]
train_x = train_x[:,permutation[valid_size+test_size:]]
train_y = train_y[:,permutation[valid_size+test_size:]]

Missing features in 687 out of 2535 samples (27.10)
Read 2534 lines
Imbalance: 6.31%
Total number of training sequences: 32


In [44]:
y

array([0, 0, 0, ..., 0, 0, 0])